In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [15]:
# Step 1: Load and Preprocess Data
# Load data from CSV
csv_file_path = 'hand_gesture_data.csv'
data = pd.read_csv(csv_file_path)

In [16]:
# Separate features and labels
X = data.drop('label', axis=1).values
y = data['label'].values

In [17]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [18]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation='softmax')  # Number of classes
])


In [20]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
# Train the model
history = model.fit(X_train, y_train, epochs=600, validation_data=(X_test, y_test))

Epoch 1/600
20/20 [==============================] - 1s 14ms/step - loss: 1.9695 - accuracy: 0.3484 - val_loss: 1.8473 - val_accuracy: 0.4750
Epoch 2/600
20/20 [==============================] - 0s 4ms/step - loss: 1.7370 - accuracy: 0.4938 - val_loss: 1.5721 - val_accuracy: 0.4500
Epoch 3/600
20/20 [==============================] - 0s 4ms/step - loss: 1.4564 - accuracy: 0.5125 - val_loss: 1.2727 - val_accuracy: 0.5125
Epoch 4/600
20/20 [==============================] - 0s 4ms/step - loss: 1.1881 - accuracy: 0.6016 - val_loss: 1.0493 - val_accuracy: 0.7125
Epoch 5/600
20/20 [==============================] - 0s 4ms/step - loss: 0.9979 - accuracy: 0.6687 - val_loss: 0.9255 - val_accuracy: 0.7563
Epoch 6/600
20/20 [==============================] - 0s 4ms/step - loss: 0.8975 - accuracy: 0.6922 - val_loss: 0.8029 - val_accuracy: 0.9563
Epoch 7/600
20/20 [==============================] - 0s 5ms/step - loss: 0.8196 - accuracy: 0.7031 - val_loss: 0.7088 - val_accuracy: 0.9563
Epoch 8/600


In [22]:
# Step 3: Save the Model in HDF5 Format
hdf5_model_path = 'hand_gesture_model.h5'
model.save(hdf5_model_path)
print(f"Model saved to {hdf5_model_path}")

Model saved to hand_gesture_model.h5


C:\Users\thoma\anaconda3\envs\mediapipe\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
# Step 4: Convert and Save the Model in TFLite Format
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\thoma\AppData\Local\Temp\tmpkmx9c57o\assets


INFO:tensorflow:Assets written to: C:\Users\thoma\AppData\Local\Temp\tmpkmx9c57o\assets


In [24]:
# Save the TFLite model
tflite_model_path = 'hand_gesture_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
print(f"TFLite model saved to {tflite_model_path}")

TFLite model saved to hand_gesture_model.tflite


In [25]:
# Optional: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

5/5 - 0s - loss: 0.0112 - accuracy: 0.9937 - 29ms/epoch - 6ms/step

Test accuracy: 0.9937499761581421


In [26]:
# quantize tflite
import tensorflow as tf
import numpy as np
import pandas as pd

# Load your pre-trained Keras model
model = tf.keras.models.load_model('hand_gesture_model.h5')

# Load your dataset
csv_file_path = 'hand_gesture_data.csv'
data = pd.read_csv(csv_file_path)
X = data.drop('label', axis=1).values

# Representative dataset generator
def representative_data_gen():
    for i in range(100):
        input_data = X[i].reshape(1, -1).astype(np.float32)
        yield [input_data]

# Convert the model to a TFLite model with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert and save the quantized model
tflite_model_quant = converter.convert()
with open('hand_gesture_model_quantized.tflite', 'wb') as f:
    f.write(tflite_model_quant)

print("Model quantization complete and saved as 'hand_gesture_model_quantized.tflite'")


INFO:tensorflow:Assets written to: C:\Users\thoma\AppData\Local\Temp\tmp6w62jxk7\assets


INFO:tensorflow:Assets written to: C:\Users\thoma\AppData\Local\Temp\tmp6w62jxk7\assets


Model quantization complete and saved as 'hand_gesture_model_quantized.tflite'


C:\Users\thoma\anaconda3\envs\mediapipe\Lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
